# Denison CS-181/DA-210 Homework

---

## Database Programming II

In [29]:
import os
import sys
import lxml
import pandas as pd
import sqlalchemy as sa

from IPython.display import Markdown as md
from IPython.display import Image

def add_modules():
    """
    Starting at the current directory and proceeding up the file system
    tree, search for a directory named `modules`.  If found, and if not
    already there, add to the Python module search path.
    
    Params: None
    
    Return: None
    """
    directory = "."
    levels = 0
    while not os.path.isdir(os.path.join(directory, "modules")) and \
          levels < 5:
        directory = os.path.join(directory, "..")
        levels += 1
    module_path = os.path.abspath(os.path.join(directory, "modules"))
    if os.path.isdir(module_path):
        if not module_path in sys.path:
            sys.path.append(module_path)

add_modules()
import util

datadir = util.resolve_dir("dbfiles")
figsdir = util.resolve_dir("figs")

### Connection Review

**Practice 1:** Fill in the body of this function:

    sqlite_connect(datadir, database)
    
which performs the common setup functionality from the connection setup steps of yesterday's notebook, namely:

1. Create and verify a path to the database file comprised of the data directory, and the name of the database with a `".db"` appended.
2. Create an sqlite connection string with `sqlite:///` and then the path from step 1.
3. Create an engine object using `create_engine()` from the `sqlalchemy` (i.e. `sa`) module.  Note that the import with alias for the package is already done for you in the prologue step.
4. Create a connection object by invoking the `connect()` method of the engine object.

The function should return the tuple of the engine and the connection object, to be used in subsequent operations, or (None, None) if the path does not exist as a file.

In [30]:
def sqlite_connect(datadir, database):
    # 1. Create path
    # 2. Create connection string
    # 3. Call create_engine()
    # 4. Call connect()
    
    path = os.path.join(datadir, database + ".db")
    assert os.path.isfile(path)
    cstring = f"sqlite:///{path}"
    engine = sa.create_engine(cstring)
    conn = engine.connect()
    
    return engine, conn
    

In [31]:
engine, conn = sqlite_connect(datadir, "book")

In [32]:
# Testing cell
assert engine is not None
assert conn is not None
assert isinstance(conn, sa.engine.base.Connection)
assert not conn.closed

In [33]:
def sqlite_teardown(engine, connection):
    try: 
        if isinstance(connection, sa.engine.base.Connection):
            connection.close()
        del connection
    except:
        pass
    try:
        del engine
    except:
        pass

In [34]:
#sqlite_teardown(engine, conn)

### Function to Query Full Table

**Practice 2:** Given what we learned yesterday, let us create a function:

    get_table(connection, table)

that obtains all rows and all columns from the table passed as a parameter.  This example both reinforces the idea of abstracting a set of steps in a function, along with the idea we want to develop further today of incorporating the values of Python variables (the parameter `table`, in this case) into an SQL query we wish to construct.

Steps:

1. Use a Python format string to create the query string.
2. Execute the query and return a result proxy
3. Accumulate the rows of the result into a list (of tuples)
    - Use a while loop, with the result of fetchone() returning None when there are no more rows to fetch.
4. Build and return a DataFrame

In [35]:
def get_table(connection, table):
    query = f"SELECT * FROM {table}"
    rproxy = connection.execute(query)
    L = rproxy.keys()
    LoT = []
    arow = rproxy.fetchone()
    while(arow != None):
        LoT.append(arow)
        arow = rproxy.fetchone()
    
    df = pd.DataFrame(LoT, columns = L)
    return df

In [36]:
df = get_table(conn, "topnames")
df.head()

,year,sex,name,count
0,1880,Female,Mary,7065
1,1880,Male,John,9655
2,1881,Female,Mary,6919
3,1881,Male,John,8769
4,1882,Female,Mary,8148


**Practice 3:** One of the great things about packages that are the most dominant in an area is that we, as programmers, can rely on them and they can "know" about each other.

- `pandas` "knows" about `sqlalchemy` and has a DataFrame constructor function that can take the string of a query, along with a connection object, and take care of all the steps to  create a DataFrame.
- `sqlalchemy` "knows" about `pandas` and has a function to use a table or a result proxy to create a data frame.

In the (global) cell that follows, create a query string of your choosing, then invoke `read_sql_query()` of `pd`, passing the query as the first argument, and using a named argument of `con=` with the Python variable for your connection.   Assign to `df`.

In [37]:
df = pd.read_sql_query("SELECT DISTINCT name FROM topnames", con=conn)
df

,name
0,Mary
1,John
2,Robert
3,James
4,Linda
5,Michael
6,David
7,Lisa
8,Jennifer
9,Jessica


### More Incorporating Variables

**Practice 4:** Let us assume a variable `lowlife`, that contains a numeric value such that, based on that value, we want to select the set of rows from `indicators0` whose value of the `life` column is greater than `lowlife`.

**Use a Python Format String** to create the variable `query` whose contents will be the SQL to perform this query and to incorporate `lowlife`.  Test it out by performing an execute() on the connection.

This time, retrieve the results by invoking the `fetchall()` method on the result proxy, and assign to variable `allrows`.

In [38]:
lowlife = 70

query = f"SELECT * FROM indicators0 WHERE life > {lowlife}"

print("Query:", query)

rproxy = conn.execute(query)
allrows = rproxy.fetchall()
print(allrows)

df = pd.DataFrame(allrows, columns=rproxy.keys())
df

Query: SELECT * FROM indicators0 WHERE life > 70
[('CHN', 1386.4, 12143.5, 76.4, 1469.88), ('FRA', 66.87, 2586.29, 82.5, 69.02), ('GBR', 66.06, 2637.87, 81.2, 79.1), ('USA', 325.15, 19485.4, 78.5, 391.6)]


,code,pop,gdp,life,cell
0,CHN,1386.40,12143.50,76.4,1469.88
1,FRA,66.87,2586.29,82.5,69.02
2,GBR,66.06,2637.87,81.2,79.10
3,USA,325.15,19485.40,78.5,391.60


In [23]:
sqlite_teardown(engine, conn)
engine, school_conn = sqlite_connect(datadir, "school")

**Practice 5:** Consider a query of the form

```sql
SELECT student_name, studentmajor
FROM students
WHERE studentlast BETWEEN low AND high
```
where we compose student_name from studentlast and studentfirst, and want to incorporate values of variables for low and high.

Complete the following function, where we pass as parameters the low and high (as well as the connection) to make that query. 

In [47]:
def student_range(conn, low, high):
    query = f"""
    SELECT studentlast || ", " || studentfirst AS student_name, studentmajor
    FROM students
    WHERE studentlast BETWEEN '{low}' AND '{high}'
    """
    
    rproxy = conn.execute(query)
    allrows = rproxy.fetchall()
    df = pd.DataFrame(allrows, columns = rproxy.keys())
    return df

In [50]:
table = student_range(school_conn, 'Pq', 'Ps')
table

,student_name,studentmajor
0,"Price, Charlie",PSYC
1,"Price, Elizabeth",ECON
2,"Price, Olivia",None
3,"Price, Kathleen",ARTS
4,"Price, Millie",ENVS
5,"Price, Stephanie",BIOL
